In [1]:
import sklearn
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K

print ( "sklearn: {}".format(sklearn.__version__) )
print ( "tensorflow: {}".format(tf.__version__) )

import sys
sys.path

sklearn: 0.21.2
tensorflow: 1.14.0


Using TensorFlow backend.


['/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/python',
 '/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib',
 '',
 '/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib/python3.7/site-packages',
 '/cvmfs/sft.cern.ch/lcg/releases/LCG_97python3/Python/3.7.6/x86_64-centos7-gcc8-opt/lib/python37.zip',
 '/cvmfs/sft.cern.ch/lcg/releases/LCG_97python3/Python/3.7.6/x86_64-centos7-gcc8-opt/lib/python3.7',
 '/cvmfs/sft.cern.ch/lcg/releases/LCG_97python3/Python/3.7.6/x86_64-centos7-gcc8-opt/lib/python3.7/lib-dynload',
 '/eos/home-a/antoniov/SWAN_projects/env/uproot4/lib/python3.7/site-packages',
 '/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib/python3.7/site-packages/IPython/extensions',
 '/scratch/antoniov/.ipython']

In [ ]:
config = tf.ConfigProto(device_count={"CPU": 12})
sess = tf.Session(config=config)

In [ ]:
sess.list_devices()

In [ ]:
K.set_session( sess )

In [ ]:
sess is K.get_session()